In [1]:
from neighbor_server import *
from social_graph import *
from shortest_paths import *

In [2]:
sg = SocialGraph("train")

Initializing check ins
Loading from Training Pickle File
Initializing network


In [3]:
from __future__ import print_function

In [4]:
'''
when you add neighbors into the graph
their g(source) = dist of last check in from source
and their h(n) = dist of last check in from end

g is cost of geographic movement from source
h becomes hueristic to choose the closest neighbor to goal

to throttle the API,
pick the top k nodes at each step that have f = g + h the minimum
rather than putting all nodes in the priority queue

otherwise put all the nodes in the queue, hoping for hueristic to take care of the problem
also maintain the got_neighbors_for set so that multiple test cases can use the same graph
'''

'\nwhen you add neighbors into the graph\ntheir g(source) = dist of last check in from source\nand their h(n) = dist of last check in from end\n\ng is cost of geographic movement from source\nh becomes hueristic to choose the closest neighbor to goal\n\nto throttle the API,\npick the top k nodes at each step that have f = g + h the minimum\nrather than putting all nodes in the priority queue\n\notherwise put all the nodes in the queue, hoping for hueristic to take care of the problem\nalso maintain the got_neighbors_for set so that multiple test cases can use the same graph\n'

In [6]:
def get_dist(source, dest, dist_fn):
    return dist_fn(sg.get_location(source), sg.get_location(dest))

In [26]:
import sys

In [43]:
def astar_temp(source, dest, got_neighbors_for, testcase_id=1, g_fn=dists.manhattan, h_fn=dists.manhattan):
    closed_set = set()
    # used to reconstruct path after finding destination
    came_from = {}
    g = {}
    for node in sg.node_checkins:
        g[node] = sys.maxsize
    g[source] = 0
    fq = Queue.PriorityQueue()
    
    # initialize heap with start node
    fq.put((get_dist(source, dest, h_fn), source))
    while not fq.empty():
        min_f, current = fq.get()
        
        if current == dest:
            print("Destination Reached")
            return (reconstructed_path(came_from, current), got_neighbors_for)
        
        closed_set.add(current)
        
        neighbors = None
        if current not in got_neighbors_for: 
            print("Getting neighbors for:", current)
            neighbors = get_neighbors(current, testcase_id)
            got_neighbors_for.add(current)
        else:
            neighbors = sg.get_neighbors(current)
        
        for neighbor in neighbors.iterkeys():
            if neighbor not in closed_set:
                tentative_g = g[current] + get_dist(current, neighbor, g_fn)
                neighbor_h = get_dist(neighbor, dest, h_fn)
                if tentative_g < g[neighbor]:
                    #open_set.add(neighbor)
                    g[neighbor] = tentative_g
                    f_score  = g[neighbor] + neighbor_h
                    fq.put((f_score, neighbor))
                    came_from[neighbor] = current
                    
def reconstructed_path(came_from, current):
    path = [current]
    while current in came_from:
        current = came_from[current]
        path.append(current)
    return [p for p in reversed(path)]


In [4]:
shortest_path, path_length, got_neighbors_for = astar_distance_based(sg, 5555, 2222, set(), "train", g_fn=dists.unweighted, h_fn=dists.l2)

Getting neighbors for: 5555
Getting neighbors for: 648
Getting neighbors for: 12614
Getting neighbors for: 647
Getting neighbors for: 649
Getting neighbors for: 9321
Getting neighbors for: 18163
Getting neighbors for: 36559
Getting neighbors for: 280
Getting neighbors for: 9327
Getting neighbors for: 40345
Getting neighbors for: 9324
Getting neighbors for: 9322
Getting neighbors for: 9323
Getting neighbors for: 32515
Getting neighbors for: 7576
Getting neighbors for: 11576
Getting neighbors for: 41857
Getting neighbors for: 15665
Getting neighbors for: 13711
Getting neighbors for: 964
Getting neighbors for: 22591
Getting neighbors for: 21373
Getting neighbors for: 2202
Getting neighbors for: 11287
Getting neighbors for: 13394
Getting neighbors for: 15947
Getting neighbors for: 37713
Getting neighbors for: 6883
Getting neighbors for: 20287
Getting neighbors for: 25511
Getting neighbors for: 17599
Getting neighbors for: 11592
Getting neighbors for: 40568
Getting neighbors for: 11290
Gett

In [9]:
shortest_path

[5555, 648, 30447, 2222]

In [ ]:
shortest_path, path_length, got_neighbors_for = astar_distance_based(sg, 5555, 2222, set(), "train", g_fn=dists.unweighted, h_fn=dists.l2)

In [13]:
shortest_path, path_length, got_neighbors_for = astar_distance_based(sg, 939, 401, set(), "train", g_fn=dists.unweighted, h_fn=dists.manhattan)

Destination Reached


In [14]:
shortest_path

[939, 5294, 27493, 10278, 401]

In [15]:
path_length

4

In [18]:
shortest_path, path_length, got_neighbors_for = astar_distance_based(sg, 5555, 2222, set(), "train", g_fn=dists.manhattan, h_fn=dists.manhattan)

Destination Reached


In [19]:
shortest_path

[5555, 8, 5187, 19390, 2222]